# Quotation Extraction: Version 3
## 0 Imports

In [1]:
import numpy
import csv
# spaCy
import spacy
# spaCy Visualizer
from spacy import displacy
from spacy.matcher import Matcher

## 1 Text processing
### 1.1 Loading and cleaning the text file
Loads the article, removes all new line characters and replaces all variants of quotes by a unique one. Puts the text through the NLP pipeline.

I have also noticed that the model doesn't deal with ";" symbols to seperate sentences, which it sometimes treats as the end of the sentence and sometimes not. I replace them with commas (",").

In [2]:
quotes = ["«", "»", "“", "”", "„", "‹", "›", "‟", "〝", "〞"]

# Replace all formats of quotation marks by the quotation mark <">
def normalize_quotes(text):
    for q in quotes:
        text = text.replace(q, '"')
    return text


with open('../data/article01.txt', 'r') as file:
    text = normalize_quotes(file.read().replace('\n', ' '))
    text = text.replace(";", ",")

nlp = spacy.load("fr_core_news_md")
doc = nlp(text)

## 2 Loading Model Parameters
### 2.1 Loading cue verbs

In [3]:
with open('../data/cue_verbs.csv', 'r') as f:
    reader = csv.reader(f)
    cue_verbs = set(list(reader)[0])

print(cue_verbs)

{'annoncer', 'abonder', 'ricaner', 'considérer', 'noter', 'sourir', 'expliquer', 'reconnaître', 'suggérer', 'déclarer', 'ajouter', 'souvenir', 'démontrer', 'lâcher', 'prôner', 'concevoir', 'relever', 'exprimer', 'crier', 'élaborer', 'définir', 'décrire', 'proposer', 'recenser', "s'interroger", 'écrire', 'dire', 'estimer', 'remarquer', "mettre l'accent sur", 'prouver', 'observer', 'déplorer', 'donner', 'soutenir', 'prétendre', 'étudier', 'mettre en évidence', 'affirmer', 'calculer', 'présenter', 'établir', 'commenter', 'poursuivre', 'proclamer', 'analyser', 'insister', 'adopter', 'corroborer', 'préciser', 'dénoncer', 'souligner', 'répondre', 'formuler', 'découvrir', 'se souvenir', 'identifier', 'montrer', 'citer', 'raconter', 'constater', 'confirmer', 'murmurer', 'résumer', 'rapporter'}


### 2.2 Loading Quotations Structures
The quotation structures are stored in a CSV file, respecting the following format. All Part-Of-Speech elements are abbreviated as follows.
* RS: Reported Speech
* CV: Cue Verb
* QT: Quotee
* text: distinct words
One structure per line, with each element seperated by a comma.

In [4]:
with open('../data/quote_structures.csv', 'r') as f:
    reader = csv.reader(f)
    structures = list(reader)

print(structures)

[['parataxis', 'obj', 'ancestor'], ['parataxis', 'nsubj', 'ancestor'], ['ROOT', 'obj', 'advcl']]


## 3 Finding quote containing sentences
### 3.1 Finding sentences containing direct quotes, and replacing the quotes with special characters

In [5]:
def is_quote(text):
    # Quote at least 3 tokens long
    a = len(text) >= 3
    # Non-proper noun words are not all capitalized 
    b = [word.shape_[0] == 'X' for word in text if not word.pos_ in ["PROPN", "PUNCT"]]
    return a and (False in b)

def quote_finder(text):
    matcher = Matcher(nlp.vocab, validate=True)
    # Add match ID "Quote"
    pattern = [{"TEXT": '"'}]
    matcher.add("Quote", None, pattern)

    # Find the quote matches
    matches = matcher(text)
    opening_quotes = []
    closing_quotes = []
    for match_id, start, end in matches:
        if len(opening_quotes) == len(closing_quotes):
            opening_quotes.append(start + 1)
        else:
            closing_quotes.append(start)

    quote_pos = zip(opening_quotes, closing_quotes)
    potential_quotes = []
    for (start, end) in quote_pos:
        quote = text[start:end]
        if is_quote(quote):
            potential_quotes.append((start, end))
    return potential_quotes

def encode_quotes(text):
    potential_quotes = quote_finder(text)
    encoded_text = ""
    prev_quote_end = 0
    for (start, end) in potential_quotes:
        encoded_text += text[prev_quote_end:start].text
        encoded_text += "*"
        prev_quote_end = end
    encoded_text += text[prev_quote_end:].text
    return encoded_text

encoded_text = encode_quotes(doc)
doc = nlp(encoded_text)
sentences = list(doc.sents)

In [6]:
# Returns true iff the sentence contains quotation marks.
def contains_cue(sentence):
    for token in sentence:
        if token.lemma_ in cue_verbs:
            return True


cues = []
        
for s in sentences:
    if contains_cue(s):
        cues.append(s)
        print(s, "\n")

Mais "*", observe Alexandre Roulin, de l’Université de Lausanne. 

A l’aide de caméras infrarouges, ils ont d’abord montré que les mâles rapportaient au nid, en moyenne, 4,78 proies par nuit. 

"*", souligne Luis San-Jose, premier auteur de l’étude. 

Mais quand l’effort de chasse était soutenu, tout changeait. 

"*"voler"*", raconte Alexandre Roulin. 

"*", explique le chercheur. 



### 3.2 Observing the structure of encoded sentences containing cue verbs.

In [7]:
for s in cues:
    displacy.render(s, style="dep", options={"compact": True})